In [46]:
from netCDF4 import Dataset
import numpy as np
import pandas as pd

In [24]:
ds

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    Observation Date: 2022.10.02
    Source Data Organization: National Centers for Environmental Information
    Source Data Name: AVHRR_OI_SST
    Data Interval(deg): 0.250
    30years SST Average Data Organization: National Centers for Environmental Information
    30years SST Average Data Name: AVHRR_OI_SST
    30years SST Average Start Date: 1991.01.01
    30years SST Average End date: 2020.12.31
    Observation Area: ROK
    dimensions(sizes): nlat(132), nlon(160)
    variables(dimensions): float32 LA(nlat), float32 LO(nlon), float32 YY30_AVG_SST_ANOMALY(nlat, nlon), float32 YY30_AVG_SST(nlat, nlon)
    groups: 

In [48]:
for ds_name in ['AVHRR_OI_SST', 'CMC_SST', 'DMI_SST', 'GAMSSA_SST', 'MUR_SST', 'MW_IR_OI_SST', 'MW_OI_SST', 'NAVO_SST', 'OSPO_SST', 'OSTIA_SST']:
    
    ds_anomaly = Dataset(f'/Volumes/T7/output/{ds_name}/nc/anomaly/2022/{ds_name}_2022_10_02_ROK_Anomaly.nc')
    ds_grade = Dataset(f'/Volumes/T7/output/{ds_name}/nc/grade/2022/{ds_name}_2022_10_02_ROK_Grade.nc')

    lat_range = ds_anomaly['LA'][:].data
    lon_range = ds_anomaly['LO'][:].data

    avg_sst = ds_anomaly['YY30_AVG_SST'][:].data
    anomaly_sst = ds_anomaly['YY30_AVG_SST_ANOMALY'][:].data
    grade = ds_grade['SST_GRD_CD'][:].data
    obs_date = ds_anomaly.getncattr('Observation Date')
    organization = ds_anomaly.getncattr('Source Data Organization')
    data_name = ds_anomaly.getncattr('Source Data Name')
    grid = ds_anomaly.getncattr('Data Interval(deg)')
    yy_organization = ds_anomaly.getncattr('30years SST Average Data Organization')
    yy_data_name = ds_anomaly.getncattr('30years SST Average Data Name')
    start_date = ds_anomaly.getncattr('30years SST Average Start Date')
    end_date = ds_anomaly.getncattr('30years SST Average End date')
    obs_area = ds_anomaly.getncattr('Observation Area')

    data_stack = []

    for i in range(len(lat_range)):
        for j in range(len(lon_range)):
            if anomaly_sst[i][j] == -32768 or anomaly_sst[i][j] == 32767 or np.isnan(anomaly_sst[i][j]) :
                continue
            else :
                avg = avg_sst[i][j]
                anom = anomaly_sst[i][j]
                grd = grade[i][j]
                lat = lat_range[i]
                lon = lon_range[j]

            data_stack.append((avg, anom, grd, lat, lon, obs_date, organization, data_name, grid, yy_organization, yy_data_name, start_date, end_date, obs_area))

    df = pd.DataFrame(data_stack, 
                      columns=['YY30_AVG_SST', 'YY30_AVG_SST_ANOMALY', 'SST_GRD_CD', 'LA', 'LO', 
                               'WTCH_YMD', 'SRC_DATA_INST_NM', 'SRC_DATA_NM', 'SRC_DATA_INTV',
                               'YY30_AVG_SST_SRC_DATA_INST_NM', 'YY30_AVG_SST_SRC_DATA_NM', 
                               'YY30_SYMD', 'YY30_EYMD', 'WTCH_RELM_AREA'])

    df.to_csv(f'/Volumes/T7/csv_table/{ds_name}.csv')